# Inlämning 2: Numpy, Pandas och Analys

### Part 1: Numpy

In [ ]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)

Konstruera en array med 10 kolumner och 5 rader av float-tal. Inget tal får vara samma.

In [ ]:
first_array = np.random.normal(size=(5, 10))

print(first_array)
print(first_array.dtype)

Konstruera en ny array till med 10 kolumner och 7 rader av float-tal. Den skall inte vara samma som ovan array..

In [ ]:
second_array = np.random.randn(7, 10)

print(second_array)
print(second_array.dtype)

Gör om de ovan arrayerna så att de har 5 kolumner men innehåller samma data.

In [ ]:
print(first_array)
print(second_array)
first_array = first_array.reshape(10,5)
second_array = second_array.reshape(14,5)

print(first_array)
print(second_array)


Slå ihop de bägge arrayerna till en array.

In [ ]:
result_array = np.concatenate((first_array, second_array), axis=0)
print(result_array)

Ändra ordningen på kolumnerna så att de har ordningen kolumn5, kolumn3, kolumn4, kolumn1 och kolumn2.

In [ ]:
out = result_array[:, [4, 2, 3, 0, 1]]
print(out)

### Part 2: Pandas

In [ ]:
import pandas as pd

In [ ]:
assaults = pd.read_csv('assaults.csv')
# print(assaults.describe())

##### 1 - Hur många regioner finns med i data?

In [ ]:
unique_region_labels = assaults['Region_2013_label'].unique()
print(f'The total number of regions is: {len(unique_region_labels)}')

##### 2 - Lista regionerna i bokstavsordning

In [ ]:
region_names = unique_region_labels.tolist()
print(sorted(region_names))

##### 3 - Vad är invånarantalet i varje region

In [ ]:
#assaults.head(20)
print('Population in mid point 2015:\n')
series_grouped = assaults.groupby('Region_2013_label')['Population_mid_point_2015'].sum()
series_grouped

##### 4 - Hur många brott begicks i varje region

In [ ]:
# assaults.head(20)
print('Total number of victims in 2015:\n')
series_grouped = assaults.groupby('Region_2013_label')['Victimisations_calendar_year_2015'].sum()
series_grouped

##### 5 - Skapa en dataframe på de områden med de 10 högsta antalet överfall.

In [ ]:
# assaults.head()
series_grouped = assaults.groupby('Area_unit_2013_label')['Victimisations_calendar_year_2015'].sum()
df_grouped = series_grouped.to_frame(name='Number_of_attacks').reset_index()
df_grouped = df_grouped.sort_values(['Number_of_attacks'], ascending=False).groupby('Area_unit_2013_label').head(10)
df_grouped = df_grouped.head(10).reset_index()
print(type(df_grouped))
df_grouped


##### 6 - Lägg till en kolumn i dataframen som visar % antal överfall i området jämfört med invånarantal i området

In [ ]:
#assaults.head()
def condition(x):
    if x['Victimisations_calendar_year_2015'] != 0 and x['Population_mid_point_2015'] !=0:
        return x['Victimisations_calendar_year_2015'] / x['Population_mid_point_2015'] * 100   
    else:
        return 0

assaults_copy = assaults
assaults_copy['Procent_attacks_by_area'] = assaults_copy.apply(condition, axis=1)
assaults_copy.head()

##### 7 - Skapa en ny dataframe som innehåller antalet områden som finns av de olika typerna av urban area. Ledtråd: groupby

In [ ]:
# assaults.head()
series_grouped = assaults.groupby('Urban_area_type').size()
df_grouped = series_grouped.to_frame(name='Number_of_areas').reset_index()
print(df_grouped)

##### 8 - Ta reda på hur många områden hade population men inga överfall. Jämför totala populationen i dessa med totala populationen i deras region.

In [ ]:
assaults_copy = assaults
assaults_copy = assaults_copy[(assaults_copy['Population_mid_point_2015'] > 0)]
assaults_copy = assaults_copy[(assaults_copy['Victimisations_calendar_year_2015'] == 0)].reset_index()
len(assaults_copy)

In [ ]:
assaults_copy = assaults
assaults_copy = assaults_copy[(assaults_copy['Population_mid_point_2015'] > 0)]
assaults_copy = assaults_copy[(assaults_copy['Victimisations_calendar_year_2015'] == 0)].reset_index()
assaults_copy['total_by_region'] = assaults_copy['Population_mid_point_2015'].groupby(assaults_copy['Region_2013_label']).transform('sum')
df_population_comparison = assaults_copy[['Area_unit_2013_label','Population_mid_point_2015', 'Region_2013_label', 'total_by_region']]
df_population_comparison = df_population_comparison.assign(Procent_region_area=lambda x: x['Population_mid_point_2015'] / x['total_by_region'] * 100)
df_population_comparison.head(10)

##### 9 - Räkna antalet områden som inte har någon data på överfall eller invånare.

In [ ]:
assaults_copy = assaults
assaults_copy = assaults_copy[(assaults_copy['Population_mid_point_2015'] == 0) | (['Victimisations_calendar_year_2015'] == 0)]
print(f'Antalet områden som inte har någon data på överfall eller invånare: {len(assaults_copy)}')


##### 10 - VG: Skapa en ny dataframe med 10 områderna med mer än 5000 invånare, men har de lägsta överfallen per invånare.

In [ ]:
assaults_copy = assaults
assaults_copy = assaults_copy.loc[(assaults_copy['Population_mid_point_2015'] > 5000) & (assaults_copy['Victimisations_calendar_year_2015'] > 0)]
assaults_copy = assaults_copy.sort_values(by=assaults_copy.columns[5], ascending=True).head(10)
assaults_copy = assaults_copy.iloc[:,1:6]
assaults_copy = assaults_copy.reset_index()
assaults_copy

##### 11 - VG: Skapa en ny dataframe som innehåller % överfall i regionen jämfört med antalet invånare i regionen.

In [ ]:
assaults_copy = assaults
series_grouped_attacks = assaults_copy.groupby('Region_2013_label')['Victimisations_calendar_year_2015'].sum()
series_grouped_population = assaults_copy.groupby('Region_2013_label')['Population_mid_point_2015'].sum()
data_in_dictionary = {'Attacks_by_region': series_grouped_attacks, 'Population_by_region': series_grouped_population }
dataframe = pd.DataFrame(data_in_dictionary)
dataframe.index.name =''
dataframe = dataframe.assign(Procent_attacks_by_region=lambda x: x['Attacks_by_region'] / x['Population_by_region'] * 100)
dataframe


##### 12 - VG: Lägg till en ny kolumn i dataframen som visar % antal överfall per region

In [ ]:
def condition(x):
    series_grouped_attacks = assaults.groupby('Region_2013_label')['Victimisations_calendar_year_2015'].sum()
    series_grouped_population = assaults.groupby('Region_2013_label')['Population_mid_point_2015'].sum()
        
    if x['Region_2013_label'] == 'Area Outside Region':
        return series_grouped_attacks[0] / series_grouped_population[0] * 100
    elif x['Region_2013_label'] == 'Auckland Region':
        return series_grouped_attacks[1] / series_grouped_population[1] * 100
    elif x['Region_2013_label'] == 'Bay of Plenty Region':
        return series_grouped_attacks[2] / series_grouped_population[2] * 100
    elif x['Region_2013_label'] == 'Canterbury Region':
        return series_grouped_attacks[3] / series_grouped_population[3] * 100    
    elif x['Region_2013_label'] == 'Gisborne Region':
        return series_grouped_attacks[4] / series_grouped_population[4] * 100
    elif x['Region_2013_label'] == 'Hawke\'s Bay Region':
        return series_grouped_attacks[5] / series_grouped_population[5] * 100
    elif x['Region_2013_label'] == 'Manawatu-Wanganui Region':
        return series_grouped_attacks[6] / series_grouped_population[6] * 100
    elif x['Region_2013_label'] == 'Marlborough Region':
        return series_grouped_attacks[7] / series_grouped_population[7] * 100
    elif x['Region_2013_label'] == 'Nelson Region':
        return series_grouped_attacks[8] / series_grouped_population[8] * 100
    elif x['Region_2013_label'] == 'Northland Region':
        return series_grouped_attacks[9] / series_grouped_population[9] * 100
    elif x['Region_2013_label'] == 'Otago Region':
        return series_grouped_attacks[10] / series_grouped_population[10] * 100
    elif x['Region_2013_label'] == 'Southland Region':
        return series_grouped_attacks[11] / series_grouped_population[11] * 100
    elif x['Region_2013_label'] == 'Taranaki Region':
        return series_grouped_attacks[12] / series_grouped_population[12] * 100
    elif x['Region_2013_label'] == 'Tasman Region':
        return series_grouped_attacks[13] / series_grouped_population[13] * 100
    elif x['Region_2013_label'] == 'Waikato Region':
        return series_grouped_attacks[14] / series_grouped_population[14] * 100
    elif x['Region_2013_label'] == 'Wellington Region':
        return series_grouped_attacks[15] / series_grouped_population[15] * 100
    elif x['Region_2013_label'] == 'West Coast Region':
        return series_grouped_attacks[16] / series_grouped_population[16] * 100

assaults_copy = assaults
assaults_copy = assaults_copy[assaults_copy['Victimisations_calendar_year_2015'] !=0]
assaults_copy = assaults_copy[assaults_copy['Population_mid_point_2015'] != 0]
assaults_copy['Procent_attacks_by_region'] = assaults_copy.apply(condition, axis=1)
assaults_copy.head()

### Part 3: Utforskning/analys

Minst 500 rader, 5 Kolumner i ditt valda dataset

#### Step 1: Välj dataset

Hitta ett intressant dataset. Det valda datasettet måste vara i CSV format,  ha minst 500 rader och 5 kolumner. 
Ladda ner datasetet med pandas read_csv funktion och en url (se nedan). Alternativt i inlämningen skicka med datasettet.

Link to the dataset in Kaggle: https://www.kaggle.com/datasets/prosperchuks/health-dataset

In [ ]:
import pandas as pd
health_dataframe = pd.read_csv('health_data.csv')

#### Step 2: Data preparation och Cleaning

Ladda in dataset in i en dataframe med pandas. 
Behandla missade data, icke-korrekt data och fel data.
Gör andra steg som behövs (gör om sträng datum till riktiga datum, skapa fler kolumner, slå ihop datasets osv).
Summera hur datasetet ser ut i nuvarande läge i dina markdowns. Storlek, kolumner, kategori typer (qualitative vs. quantitative), kvalitet på data, fång osv.

Info about the data: There are 18 feature variables and 3 target variables, Diabetes, Hypertension, and Stroke, each target variable having 2 classes.

Showing the first 5 rows of the dataset:

In [ ]:
health_dataframe.head(5)

Exploring the number of dimensions and the datatypes

In [ ]:
print(f'ndim: {health_dataframe.ndim}')

Analysing some statistics of the data

In [ ]:
print(health_dataframe.describe())

Checking for null data

In [ ]:
health_dataframe.isnull().sum()

Checking for NaN

In [ ]:
health_dataframe.isna().sum()

Removing duplicates

In [ ]:
health_dataframe.drop_duplicates()

Checking the datatypes

In [ ]:
health_dataframe.dtypes

Converting datatypes to int

In [ ]:
health_dataframe = health_dataframe.astype(int)
health_dataframe.dtypes

Checking the unique values on the dataset in order to see if there are missing values or strange values

In [ ]:
print(f'The different age groups: {sorted(health_dataframe["Age"].unique())}')
print(f'The different genders: {sorted(health_dataframe["Sex"].unique())}')
print(f'The different high cholesterol values: {sorted(health_dataframe["HighChol"].unique())}')
print(f'The different cholesterol check in 5 years values: {sorted(health_dataframe["CholCheck"].unique())}')
print(f'The different "is a smoker" values: {sorted(health_dataframe["Smoker"].unique())}')
print(f'The different "heart condition" values: {sorted(health_dataframe["HeartDiseaseorAttack"].unique())}')
print(f'The different "practices physcical activity" groups: {sorted(health_dataframe["PhysActivity"].unique())}')
print(f'The different "Eat fruits once a day" values: {sorted(health_dataframe["Fruits"].unique())}')
print(f'The different "Eat vegetables once a day" values: {sorted(health_dataframe["Veggies"].unique())}')
print(f'Adult male - more than 14/week drinks & adult female - more than 7 drinks/week: {sorted(health_dataframe["HvyAlcoholConsump"].unique())}')
print(f'The "Good health" on a scale 1-5: {sorted(health_dataframe["GenHlth"].unique())}')
print(f'The "Days of poor mental healt" on a scale 1-30: {sorted(health_dataframe["MentHlth"].unique())}')
print(f'The "physical illness or injury days in past 30 days" scale 1-30: {sorted(health_dataframe["PhysHlth"].unique())}')
print(f'The "Difficult walking" values: {sorted(health_dataframe["DiffWalk"].unique())}')
print(f'Diabetes labels: {sorted(health_dataframe["Diabetes"].unique())}')
print(f'Hypertension labels: {sorted(health_dataframe["Hypertension"].unique())}')
print(f'Stroke lavels: {sorted(health_dataframe["Stroke"].unique())}')


#### Step 3: Undersökande analys och visualisering

Undersök data genom analysera (mean, sum, range osv)  intressant statistik i numeriska kolumnerna.
Undersök distributions av numeriska kolumner genom histogram osv
Undersök relationen mellan kolumner med scatter plots, bar chars osv.
Notera intressanta upptäckter i din markdown.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="darkgrid")

fig, axs = plt.subplots(2, 2, figsize=(15, 15))
sns.histplot(data=health_dataframe, x="Age", hue="Sex", multiple="stack", ax=axs[0, 0])
sns.histplot(data=health_dataframe, x="HighChol", hue="Sex", multiple="stack", ax=axs[0, 1])
sns.histplot(data=health_dataframe, x="Smoker", hue="Sex", multiple="stack", ax=axs[1, 0])
sns.histplot(data=health_dataframe, x="HeartDiseaseorAttack", hue="Sex", multiple="stack", ax=axs[1, 1])

plt.show()

Checking the correlation in the dataset. 

In [ ]:
health_dataframe.corr()

Plotting a heatmat of the correlation

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")
corr = health_dataframe.corr()
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr,  cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

#### Step 4: För VG - fråga och svara på 4 intressanta frågor om datan
##### 4.1 - Based on the heatmap plotted above, which were the features more related to the incidence of diabetes?
Answer: Age group, High cholesterol values, BMI, General health and Hypertension.
##### 4.2 - Based on the heatmap plotted above, which were the features more related to the incidence of Hypertension?
Answer: Age group, High cholesterol values, BMI, General health and Diabetes.
##### 4.3 - Based on the heatmap plotted above, which were the features more related to the incidence of Stroke?
Answer: Heart condition, General Health. Although the correlation was not so high.
##### 4.4- What can you conclude about people who did not practice physical activity, based on the heatmap above?
Answer: The physical activity feature is inversely proportional to general health, difficuly walking and "physical illness or injury days in the past 30 days". It means that when one of the features increases, the other decreases. E.g. The more the difficulty walking, the less the physical activity. 

#### Step 5: Summera dina slutsatser & Skriv ett avslut.

Diabetes, hypertension, and stroke are three health conditions that are closely related and can have serious consequences if left untreated. The data shows that there is factors such as Age group, High Cholesterol, BMI, General Health and Heart conditions can straightly be related to the development of these conditions. 